# Fun with Featurtools

In this example [featuretools](https://www.featuretools.com/) an python frame for automated feature engineering is used to generate a wide range of features for machine learning. The aim of the Notebook is to show how the framework is used and how it could support ML workflows, but should not be considered as a tutorial on Machine Learning.

One of the main benefits of the frame work is the so called deep feature synthesis (dfs), which used relations in the data for automatically generating feature, this could be, obviously depending on your data, features like mean session length, average purchasing price or standard deviation of time since last session. The term deep comes referes to stacking generated features on top of each other. 

The main building block of Featuretools are feature primiteves and can be represented in two main groups: aggregation and transformation primitive. Like the name indicates, aggregation primitives take related inputs and return a single ountput, similar to  group by aggregations known from sql. Transformation primitives on the other hand are conveting, eg timstamps into data, hour, minutes, seconds etc or calculating differences to certain events. The full list of possible primitives can be found [here](https://primitives.featurelabs.com/).


In [ ]:
import pandas as pd
import numpy as np

import featuretools as ft

from woodwork.logical_types import Categorical, Boolean, Datetime, Double

# from featuretools_sklearn_transformer import DFSTransformer => did not work
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

from datetime import datetime
from uuid import uuid4
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
import xgboost as xgb

from scipy.stats import uniform, randint


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from typing import List, Any
from enum import Enum
from dataclasses import dataclass

pd.option_context("max_columns", None)

In [ ]:
# https://github.com/IBM/telco-customer-churn-on-icp4d
URL_TO_DATA = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"


BACK_COUNT_DATE = datetime.fromisoformat("2022-01-01")
TEST_SIZE = 0.2
VALID_SIZE = 0.25
RANDOM_STATE = 42
NUMERIC_TRANSFORMER_REPLACEMENT = "median"

In [ ]:
# fmt:off
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy=NUMERIC_TRANSFORMER_REPLACEMENT)),
        ("scaler", StandardScaler()),
    ]
)
# fmt:on 

def get_cols_by_type(
    in_df: pd.DataFrame, types_to_search: list, exclude_ids: Boolean = True
) -> list:
    cols = in_df.select_dtypes(include=types_to_search).columns.to_list()

    if exclude_ids:
        return list(filter(lambda x: not x.endswith("ID"), cols))

    return cols

- ft titanic https://www.kaggle.com/code/liananapalkova/automated-feature-engineering-for-titanic-dataset/notebook
- the dataset https://github.com/IBM/telco-customer-churn-on-icp4d/blob/master/data/Telco-Customer-Churn.csv
- titanic https://medium.com/dataexplorations/tool-review-can-featuretools-simplify-the-process-of-feature-engineering-5d165100b0c3
- time indexing recomandations https://stackoverflow.com/questions/49711987/how-do-i-prevent-data-leakage-with-featuretools




### Denormalize into 
- customer_df:      customer_id (PK), subscription_id (FK), gender, SeniorCitizen, Partner, Dependents, tenure, Churn
- services_df:      service_id (PK), service_name
- subscription_df:  subscription_id (PK), service_id, customer_id
- billing_df:       billing_id, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges


In [ ]:
# in case of CERTIFICATE_VERIFY_FAILED run Install Certificates.command
# see also https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org
df = pd.read_csv(filepath_or_buffer=URL_TO_DATA, index_col=0)

In [ ]:
# TODO: write nice dataclass to map featertools type, pandas and sklearn type
# add this info also

CUSTOMERS = {
    # "customerID",
    "gender":Boolean,
    "SeniorCitizen":Boolean,
    "Partner":Boolean,
    "Dependents":Boolean,
}


SUBSCRIPTIONS = {
    # "customerID",
    "PhoneService" :Categorical,
    "MultipleLines":Boolean,
    "InternetService":Categorical,
    "OnlineSecurity":Categorical,
    "OnlineBackup":Categorical,
    "DeviceProtection":Categorical,
    "TechSupport":Categorical,
    "StreamingTV":Categorical,
    "StreamingMovies":Categorical,
}


BILLINGS = {
    # "customerID",
    "tenure":Double,
    "Contract":Categorical,
    "PaperlessBilling":Boolean,
    "PaymentMethod":Categorical,
    "MonthlyCharges":Double,
    "TotalCharges":Double,
    # "Churn",
}






In [ ]:
# convert
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# generate syntetic time index
df["ContractStartDate"] = list(
    map(
        lambda tenure, dat=BACK_COUNT_DATE: dat - relativedelta(months=-tenure),
        df["tenure"],
    )
)

# convert to categoric for pipline processing


df["customerID"] = df.index
df["billingID"] = [str(uuid4()) for _ in range(df.shape[0])]
df["subscriptionID"] = [str(uuid4()) for _ in range(df.shape[0])]

# convert to 1/0
df["Churn"] = np.where(df["Churn"] == "Yes", 1, 0)


df["ShortContract"] = df['Contract'] == 'Month-to-month'
BILLINGS['ShortContract'] = Boolean

# remove because it depends on Phone service
products = [n for n in list(SUBSCRIPTIONS.keys()) if n != 'MultipleLines']

df['TotalProductCount'] = (df[products] != 'No').sum(axis=1)
SUBSCRIPTIONS['TotalProductCount'] = Double

df['UserCategory'] = pd.cut(x=df['TotalProductCount'], bins=[0, 3, 6, np.Inf], labels=['light', 'mid', 'heavy'])
CUSTOMERS['UserCategory'] = Categorical



In [ ]:
@dataclass
class EntitySetColumns:
    new_dataframe_name: str
    index: str
    additional_columns: list


entity_set_columns = {
    "customers": EntitySetColumns(
        index="customerID", new_dataframe_name=None, additional_columns=list(CUSTOMERS.keys())
    ),
    "subscriptions": EntitySetColumns(
        index="subscriptionID",
        new_dataframe_name="subscriptions",
        additional_columns=list(SUBSCRIPTIONS.keys()),
    ),
    "billings": EntitySetColumns(
        index="billingID", new_dataframe_name="billings", additional_columns=list(BILLINGS.keys())
    ),
}

In [ ]:
y = df["Churn"]
X = df.drop(columns=["Churn"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=VALID_SIZE, random_state=RANDOM_STATE
)  # 0.25 x 0.8 = 0.2

In [ ]:
def entityset_wrapper(
    id: str,
    data_frame_name: str, 
    feature_dataset: pd.DataFrame,
    variable_types: dict,
    entity_set_columns: dict,
) -> ft.EntitySet:
    """Non generic wrapper for making ft.EntitySet"""
    es = ft.EntitySet(id=id)

    es.add_dataframe(
        dataframe_name=data_frame_name,
        index=entity_set_columns.get("customers").index,
        logical_types=variable_types,
        dataframe=feature_dataset,
    )

    es.normalize_dataframe(
        base_dataframe_name=data_frame_name,
        new_dataframe_name=entity_set_columns.get("billings").new_dataframe_name,
        index=entity_set_columns.get("billings").index,
        additional_columns=entity_set_columns.get("billings").additional_columns,
    )

    es.normalize_dataframe(
        base_dataframe_name=data_frame_name,
        new_dataframe_name=entity_set_columns.get("subscriptions").new_dataframe_name,
        index=entity_set_columns.get("subscriptions").index,
        additional_columns=entity_set_columns.get("subscriptions").additional_columns,
    )

    return es

In [ ]:
es = entityset_wrapper(
    id="customers_train",
    data_frame_name="customers",
    feature_dataset=X_train,
    variable_types=CUSTOMERS | SUBSCRIPTIONS | BILLINGS,
    entity_set_columns=entity_set_columns,
)
es.plot()

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    max_depth=2,
    include_cutoff_time=False,
    agg_primitives=None,
    cutoff_time=None,
    instance_ids=None,
    seed_features=None
)

feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)


In [ ]:
# feature selection
#feature_selector = Pipeline(
#    steps=[
#        ("rm_highly_null_features",         FunctionTransformer(remove_highly_null_features)),
#        ("rm_single_value_features",        FunctionTransformer(remove_single_value_features)),
#    ]
#)

    #remove_highly_correlated_features,
    #remove_highly_null_features,
    #remove_single_value_features,

 #   set(features) - set(new_features)

# not necessary
#tmp, temp2  = remove_highly_null_features(feature_matrix_enc, features_enc, pct_null_threshold=.90)
#set(features_enc) - set(temp2)

#feature_matrix_enc_selected, features_enc_selected = remove_highly_correlated_features(feature_matrix_enc, features=features_enc)
#set(features_enc) - set(features_enc_selected)

feature_matrix_enc_selected, features_enc_selected = remove_highly_null_features(feature_matrix_enc, features=features_enc, pct_null_threshold = .85)
set(features_enc) - set(features_enc_selected)

fm_selected, f_selected = remove_single_value_features(feature_matrix_enc_selected, features=features_enc_selected)
set(features_enc) - set(f_selected)





In [ ]:

# fmt: off
numeric_features_ft = get_cols_by_type(
    in_df=fm_selected,
    types_to_search=[np.float64, np.int64],
    exclude_ids=False,
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features_ft),
    ], verbose=True, remainder = "passthrough"
)
# fmt: on
preprocessor

In [ ]:
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.svm import LinearSVC


VARIANCE_THRESHOLD = .8 * (1- .8)

#svm.SVC()
# fmt: off
clf = Pipeline(
    steps=[ ("preprocessor", preprocessor), 
            ('feature_selection', SelectFromModel(LinearSVC(penalty="l2", max_iter = 10000))),
            ('feture_selecction_variance', VarianceThreshold(threshold=VARIANCE_THRESHOLD)),
            ("classifier", xgb.XGBClassifier(objective="binary:logistic", random_state=RANDOM_STATE))],
            verbose=True
            
)
# fmt: on

train_transformed = fm_selected


clf.fit(train_transformed, y_train)

In [ ]:
es_val = entityset_wrapper(
    id="customers_validation",
    data_frame_name="customers",
    feature_dataset=X_val,
    variable_types=CUSTOMERS | SUBSCRIPTIONS | BILLINGS,
    entity_set_columns=entity_set_columns,
)

val_transformed = ft.calculate_feature_matrix(f_selected, es_val)



es_test = entityset_wrapper(
    id="customers_test",
    data_frame_name="customers",
    feature_dataset=X_test,
    variable_types=CUSTOMERS | SUBSCRIPTIONS | BILLINGS,
    entity_set_columns=entity_set_columns,
)

test_transformed = ft.calculate_feature_matrix(f_selected, es_test)



print("model score: %.3f" % clf.score(val_transformed, y_val))
print("model score: %.3f" % clf.score(test_transformed, y_test))

## Estimating baseline Model

In [ ]:
# fmt: off
one_hot_enc = OneHotEncoder(handle_unknown="ignore")
preprocessor_bm = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, get_cols_by_type(in_df=X_train, types_to_search=[np.float64])),
        ("onehot", one_hot_enc,      get_cols_by_type(in_df=X_train, types_to_search=[np.object0, np.object_, 'category', 'bool'])),
    ]
)
# fmt: on
preprocessor_bm

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_bm = Pipeline(
     steps=[("preprocessor", preprocessor_bm), ("classifier", xgb.XGBClassifier(objective="binary:logistic", random_state=RANDOM_STATE))]
   # steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

clf_bm.fit(X_train, y_train)
print("model score: %.3f" % clf_bm.score(X_val, y_val))
print("model score: %.3f" % clf_bm.score(X_test, y_test))

In [ ]:
#inspect_data = Pipeline(steps=[("preprocessor", preprocessor)])

#df_train_ft = clf.named_steps["preprocessor"].fit_transform(train_transformed)
df_train_ft = clf.named_steps["feture_selecction_variance"].fit_transform(train_transformed)
df_train_normal = clf_bm.named_steps["preprocessor"].fit_transform(X_train)

#72

print(df_train_ft.shape)
print(df_train_normal.shape)


fm_selected.shape





In [ ]:
numeric_features_ft

In [ ]:
get_cols_by_type(in_df=X_train, types_to_search=[np.float64])
get_cols_by_type(in_df=X_train, types_to_search=[np.object0, np.object_])
in_df=X_train
types_to_search=[np.object0, np.object_]

In [ ]:
pd.DataFrame(df_train_normal)


In [ ]:
print(train_transformed.shape)

df_train_ft = clf.named_steps["preprocessor"].fit_transform(train_transformed)



In [ ]:
#(df['PhoneService'] == 'Yes')

#(df['InternetService'] == 'Yes')
#df['InternetService'].unique()


products = [n for n in list(SUBSCRIPTIONS.keys()) if n != 'MultipleLines']

res = (df[products] != 'No').sum(axis=1)


pd.cut(x=(df[products] != 'No').sum(axis=1), bins=[0, 3, 6, np.Inf], labels=['light', 'mid', 'heavy']).value_counts()


dd

In [ ]:
SEEDED_FEATURES

____

In [ ]:
fm_selected.shape